In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('data/clean/train_clean_v2.csv')
df.head()

,fecha_dato,fecha_dato_year,fecha_dato_month,fecha_dato_day,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2015-01-28,2015,1,28,1375586,1,1,1,35,2015-01-12,...,2,1,20,1,1.0,29.0,7,1.0,87218.10,1
1,2015-01-28,2015,1,28,1050611,1,1,1,23,2012-08-10,...,2,2,1,1,1.0,13.0,23,0.0,35548.74,2
2,2015-01-28,2015,1,28,1050612,1,1,1,23,2012-08-10,...,2,1,1,1,1.0,13.0,23,0.0,122179.11,2
3,2015-01-28,2015,1,28,1050613,1,1,1,22,2012-08-10,...,2,1,7,1,1.0,50.0,6,0.0,119775.54,2
4,2015-01-28,2015,1,28,1050615,1,1,1,23,2012-08-10,...,2,1,1,1,1.0,45.0,16,0.0,22220.04,2


In [ ]:
df.as_matrix().shape

In [ ]:
df_targets = pd.read_csv('data/clean/train_labels.csv')
df_targets.head()

In [ ]:
df_targets.shape

### División de datos

In [27]:
edge = df.groupby(['fecha_dato']).get_group(np.sort(list(df.groupby(['fecha_dato']).groups.keys()))[-1]).index[0]
edge

10094948

In [47]:
df.drop(['fecha_dato', 'fecha_alta'], inplace=True, axis=1)

In [49]:
train_data = df.iloc[:edge, :]
test_data = df.iloc[edge:, :]
train_target = df_targets.iloc[:edge, 0] # El target que se va usar es ind_ahor_fin_ult1
test_target = df_targets.iloc[edge:, 0]

In [50]:
train_data.shape, train_target.shape

((10094948, 25), (10094948,))

In [51]:
test_data.shape, test_target.shape

((700444, 25), (700444,))

First try!

In [52]:
from sklearn.tree import DecisionTreeClassifier

In [52]:
%%time
clf = DecisionTreeClassifier()
clf.fit(train_data, train_target)

CPU times: user 53.1 s, sys: 1.22 s, total: 54.3 s
Wall time: 54.4 s


In [53]:
clf.score(test_data, test_target)

0.99999143400471702

Se usará un árbol de decisión para esta primera prueba, además se usará la métrica por defecto del algoritmo dicho

In [56]:
%%time
for i, col in enumerate(df_targets.columns):
    train_target = df_targets.iloc[:edge, i] 
    test_target = df_targets.iloc[edge:, i]
    clf.fit(train_data, train_target)
    score = clf.score(test_data, test_target)
    print(i, col, 'el score es de', score)

0 ind_ahor_fin_ult1 el score es de 0.999991434005
1 ind_aval_fin_ult1 el score es de 0.999998572334
2 ind_cco_fin_ult1 el score es de 0.971843573505
3 ind_cder_fin_ult1 el score es de 0.999931472038
4 ind_cno_fin_ult1 el score es de 0.986511412761
5 ind_ctju_fin_ult1 el score es de 0.999890069727
6 ind_ctma_fin_ult1 el score es de 0.998656566406
7 ind_ctop_fin_ult1 el score es de 0.98982502527
8 ind_ctpp_fin_ult1 el score es de 0.995445745841
9 ind_deco_fin_ult1 el score es de 0.999915767713
10 ind_deme_fin_ult1 el score es de 0.999780139454
11 ind_dela_fin_ult1 el score es de 0.994215097852
12 ind_ecue_fin_ult1 el score es de 0.990054879476
13 ind_fond_fin_ult1 el score es de 0.99744590574
14 ind_hip_fin_ult1 el score es de 0.999217639097
15 ind_plan_fin_ult1 el score es de 0.998311071263
16 ind_pres_fin_ult1 el score es de 0.999787277784
17 ind_reca_fin_ult1 el score es de 0.992885940917
18 ind_tjcr_fin_ult1 el score es de 0.983297736864
19 ind_valo_fin_ult1 el score es de 0.99601110

**En este punto me di cuenta que no se puede hacer una predicción como se hace corrientemente, pues cada predicción debe ser específica para cada usuario. *Así que la ejecución anterior sirvió para inferir esto* **

In [13]:
group_ncodpers = df.groupby(['ncodpers'])

In [8]:
list(group_ncodpers.groups.keys())[0]

15889

In [64]:
df_group = group_ncodpers.get_group(15889).copy()

In [65]:
df_group.drop(['fecha_dato', 'fecha_alta', 'ncodpers'], axis=1, inplace=True)
df_group

,fecha_dato_year,fecha_dato_month,fecha_dato_day,ind_empleado,pais_residencia,sexo,age,fecha_alta_year,fecha_alta_month,fecha_alta_day,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
341064,2015,1,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
688575,2015,2,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
1378582,2015,3,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
1548429,2015,4,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
2590475,2015,5,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
2938335,2015,6,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
3364885,2015,7,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
4362212,2015,8,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
4686069,2015,9,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3
5681959,2015,10,28,3,1,1,56,1995,1,16,...,2,1,2,1,1.0,28.0,1,1.0,326124.9,3


In [66]:
test_df = df_group.iloc[-1:, :]
train_df = df_group.iloc[:-1, :]
train_target = df_targets.iloc[df_group.iloc[:-1, :].index]
test_target = df_targets.iloc[df_group.iloc[-1:, :].index]

In [67]:
train_df.shape, train_target.shape, test_df.shape, test_target.shape

((16, 24), (16, 24), (1, 24), (1, 24))

In [68]:
clf = DecisionTreeClassifier()
for i, col in enumerate(df_targets.columns):
    tt = train_target.iloc[:, i] #train_target
    tst = test_target.iloc[:, i] #test_target
    clf.fit(train_df, tt)
    score = clf.score(test_df, tst)
    print(i, col, 'el score es de', score)

0 ind_ahor_fin_ult1 el score es de 1.0
1 ind_aval_fin_ult1 el score es de 1.0
2 ind_cco_fin_ult1 el score es de 1.0
3 ind_cder_fin_ult1 el score es de 1.0
4 ind_cno_fin_ult1 el score es de 1.0
5 ind_ctju_fin_ult1 el score es de 1.0
6 ind_ctma_fin_ult1 el score es de 1.0
7 ind_ctop_fin_ult1 el score es de 1.0
8 ind_ctpp_fin_ult1 el score es de 1.0
9 ind_deco_fin_ult1 el score es de 1.0
10 ind_deme_fin_ult1 el score es de 1.0
11 ind_dela_fin_ult1 el score es de 1.0
12 ind_ecue_fin_ult1 el score es de 1.0
13 ind_fond_fin_ult1 el score es de 1.0
14 ind_hip_fin_ult1 el score es de 1.0
15 ind_plan_fin_ult1 el score es de 1.0
16 ind_pres_fin_ult1 el score es de 1.0
17 ind_reca_fin_ult1 el score es de 1.0
18 ind_tjcr_fin_ult1 el score es de 1.0
19 ind_valo_fin_ult1 el score es de 1.0
20 ind_viv_fin_ult1 el score es de 1.0
21 ind_nomina_ult1 el score es de 1.0
22 ind_nom_pens_ult1 el score es de 1.0
23 ind_recibo_ult1 el score es de 1.0
